# Worksheet 07

Name: Jonas Raedler <br>
UID: U06482163

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [28]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

c1 = np.random.normal(mean, stdev, 10).tolist()
print(c1)

[5.0712731466283625, 5.412816920027575, 4.515788342155412, 3.8527043636028493, 6.185782303758408, 3.7977935851258944, 5.136291111989237, 5.882815069278788, 6.332664384422059, 4.49465735036179]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [29]:
c2 = np.random.normal(8, 1, 10).tolist()
print(c2)

[7.533603765683934, 7.487827387245159, 7.027810686206974, 7.558514873231687, 6.677467943145357, 8.427601902511444, 8.704332459111278, 5.974540963960693, 5.753920666750248, 7.935965376390275]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [30]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = c1.pop()
        data.append(p1)
    else:
        p2 = c2.pop()
        data.append(p2)
print(data)

[7.935965376390275, 4.49465735036179, 6.332664384422059, 5.753920666750248, 5.974540963960693, 5.882815069278788, 8.704332459111278, 8.427601902511444, 6.677467943145357, 7.558514873231687]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

- $N_1(\mu_1, \sigma_1)$ = $N_1(5, 1)$ as well as $P(C_1) = 0.5$
- $N_2(\mu_2, \sigma_2)$ = $N_2(8, 1)$ as well as $P(C_2) = 0.5$

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(C_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(C_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [54]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

c1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(c1)
c2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(c2)

[4.49465735036179, 6.332664384422059, 5.753920666750248, 5.974540963960693, 5.882815069278788, 6.677467943145357]
[7.935965376390275, 8.704332459111278, 8.427601902511444, 7.558514873231687]


C:\Users\jonas\BU Classes\Spring 2023\CS 506\virtualenv\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [55]:
prob_c = [ len(c1) / (len(c1) + len(c2)) , len(c2) / (len(c1) + len(c2)) ]
mean = [ sum(c1)/len(c1) , sum(c2)/len(c2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, c1)) / len(c1) , sum(map(lambda x : (x - mean[1])**2, c2)) / len(c2) ]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(C_1) = 0.6,  P(C_2) = 0.4
mean_1 = 5.852677729653156,  mean_2 = 8.156603652811171
var_1 = 0.4633995500313719,  var_2 = 0.1949595834681031


Relatively close, but they are definitely not the same (variance values are quite lower and means are slightly off)

e) For each data point, compute `P(C_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [56]:
from scipy.stats import norm

prob_c0_x = [] # P(C_0 | X_i)
prob_c1_x = [] # P(C_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(C_j) already computed
        prob_c[j]

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x.append( pdf_i[0] * prob_c[0] / prob_x)
    prob_c1_x.append( pdf_i[1] * prob_c[1] / prob_x )

probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()


point =  7.935965376390275
probability of observing that point if it came from cluster 0 =  3.517199730877786e-05
probability of observing that point if it came from cluster 1 =  1.078571439267598
point =  4.49465735036179
probability of observing that point if it came from cluster 0 =  0.011750434286021162
probability of observing that point if it came from cluster 1 =  5.015646277039594e-77
point =  6.332664384422059
probability of observing that point if it came from cluster 0 =  0.5034817037886563
probability of observing that point if it came from cluster 1 =  2.019190230773357e-19
point =  5.753920666750248
probability of observing that point if it came from cluster 0 =  0.8415736524068527
probability of observing that point if it came from cluster 1 =  2.1399368291876016e-33
point =  5.974540963960693
probability of observing that point if it came from cluster 0 =  0.831643646737429
probability of observing that point if it came from cluster 1 =  1.2854163080355319e-27
point =  

These values make sense! This is what I would've expected.

f) Having computed `P(C_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(C_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [57]:
prob_c = [sum(prob_c0_x)/ len(prob_c0_x), sum(prob_c1_x)/ len(prob_c1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_c0_x, data)]) / sum(prob_c0_x), 
        sum([x[0] * x[1] for x in zip(prob_c1_x, data)]) / sum(prob_c1_x) ]
var = [ sum(prob_c0_x * (data - mean[0])**2) / sum(prob_c0_x) , sum(prob_c1_x * (data - mean[1])**2) / sum(prob_c1_x) ]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(C_1) = 0.6073821004059943,  P(C_2) = 0.3926178995940058
mean_1 = 5.87341353681595,  mean_2 = 8.167844236567381
var_1 = 0.4927159028629166,  var_2 = 0.19177703143389854


These values are very similar to the values from the last step (see below). The variance of the second component has gotten a bit closer to the original value.

In [58]:
# Values from previous step
# P(C_1) = 0.6,  P(C_2) = 0.4
# mean_1 = 5.852677729653156,  mean_2 = 8.156603652811171
# var_1 = 0.4633995500313719,  var_2 = 0.1949595834681031

g) Update `P(C_j | X_i)`. Comment on any differences or lack thereof you observe.

In [59]:
from scipy.stats import norm

prob_c0_x = [] # P(C_0 | X_i)
prob_c1_x = [] # P(C_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(C_j) already computed
        prob_c[j]

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x.append( pdf_i[0] * prob_c[0] / prob_x)
    prob_c1_x.append( pdf_i[1] * prob_c[1] / prob_x )

probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()


point =  7.935965376390275
probability of observing that point if it came from cluster 0 =  0.0001268151658795431
probability of observing that point if it came from cluster 1 =  1.0015151947507752
point =  4.49465735036179
probability of observing that point if it came from cluster 0 =  0.016142551595013148
probability of observing that point if it came from cluster 1 =  4.537863756501824e-80
point =  6.332664384422059
probability of observing that point if it came from cluster 0 =  0.5243987249524543
probability of observing that point if it came from cluster 1 =  2.71295785993392e-20
point =  5.753920666750248
probability of observing that point if it came from cluster 0 =  0.7862160239568046
probability of observing that point if it came from cluster 1 =  8.206821383687635e-35
point =  5.974540963960693
probability of observing that point if it came from cluster 0 =  0.7928044086687696
probability of observing that point if it came from cluster 1 =  8.231648321344236e-29
point =  5

The most noticable difference is for the last point, 7.55. The confidence that this point belongs to cluster 2 has gone down quite a bit. Given that it is somewhat close to cluster 1, it makes sense that the probability of it belonging to cluster 2 is not immensely high.

h) Use `P(C_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [71]:
labels = np.argmax(np.array([prob_c0_x, prob_c1_x]), axis=0)
print(labels)

[1 0 0 0 0 0 1 1 0 1]


### Clustering Aggregation

| Point | C | P |
|-------|---|---|
| A     | 0 | a |
| B     | 0 | b |
| C     | 2 | b |
| D     | 1 | c |
| E     | 1 | d |

a) Fill in the following table where for each pair of points determine whether C and P agree or disagree on how to cluster that pair.

| Pair | Disagreement |
|------|--------------|
| A  B |      ?       |
| A  C |      ?       |
| A  D |      ?       |
| A  E |      ?       |
| B  C |      ?       |
| B  D |      ?       |
| B  E |      ?       |
| C  D |      ?       |
| C  E |      ?       |
| D  E |      ?       |


As datasets become very large, this process can become computationally challenging.

b) Given N points, what is the formula for the number of unique pairs of points one can create?

Assume that clustering C clusters all points in the same cluster and clustering P clusters points as such:

| Point | P |
|-------|---|
| A     | 0 |
| B     | 0 |
| C     | 0 |
| D     | 1 |
| E     | 1 |
| F     | 2 |
| G     | 2 |
| H     | 2 |
| I     | 2 |

c) What is the maximum number of disagreements there could be for a dataset of this size? (use the formula from b)?

d) If we look at cluster 0. There are (3 x 2) / 2 = 3 pairs that agree with C (since all points in C are in the same cluster). For each cluster, determine how many agreements there are. How many total agreements are there? How many disagreements does that mean there are between C and P?

e) Assuming that filtering the dataset by cluster number is a computationally easy operation, describe an algorithm inspired by the above process that can efficiently compute disagreement distances on large datasets.